# 1. Skip-gram with naiive softmax
- Implementation model 'Skip-gram with naiive softmax'

In [24]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter

In [25]:
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [26]:
# Replacement of lambda 'flatten'
co = []
for a1 in corpus:
    for a2 in a1:
        co.append(a2)
cor = list(set(co))

In [27]:
print(torch.__version__)
print(nltk.__version__)

0.4.1
3.2.4


In [28]:
USE_CUDA = torch.cuda.is_available()
USE_CUDA

False

In [29]:
if USE_CUDA:
    FloatTensor = torch.cuda.FloatTensor 
else:
    FloatTensor = torch.FloatTensor
# Upper code can be converted more simple

In [30]:
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [31]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex : eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
        
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch


In [32]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w : word2index[w] if word2index.get(w) is not None else word2index['<UNK'], seq))
    return Variable(LongTensor(idxs))

In [33]:
def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index['<UNK>']]))
def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

## 1. Data Load
- Data : Gutenberg corpus

In [34]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [35]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:100] # 텍스트를 scentence화
corpus = [[word.lower() for word in sent] for sent in corpus] # 단어들 소문자화

## 2. Extract stopwords from unigram dstribution's tail

In [36]:
word_count = Counter(flatten(corpus))
word_count

Counter({'!': 5,
         '"': 26,
         "'": 20,
         '(': 3,
         ')': 3,
         ').': 1,
         ',': 96,
         '-': 24,
         '--': 27,
         '--(': 1,
         '.': 66,
         '."': 26,
         '...': 5,
         '1851': 1,
         '890': 1,
         ':': 1,
         ';': 12,
         '[': 1,
         ']': 1,
         'a': 21,
         'about': 1,
         'acres': 1,
         'affording': 1,
         'after': 1,
         'againe': 1,
         'against': 1,
         'alfred': 1,
         'all': 5,
         'allusions': 1,
         'almost': 1,
         'aloft': 1,
         'alone': 1,
         'also': 1,
         'altogether': 1,
         'am': 1,
         'among': 2,
         'an': 3,
         'ancient': 1,
         'and': 35,
         'anglo': 1,
         'animal': 1,
         'annals': 1,
         'any': 1,
         'anyways': 1,
         'apology': 1,
         'appeared': 1,
         'appearing': 1,
         'appears': 2,
         'arched': 1,
      

In [37]:
border = int(len(word_count) * 0.01)
border

5

In [38]:
word_count.most_common()[:border]

[(',', 96), ('.', 66), ('the', 58), ('of', 36), ('and', 35)]

In [39]:
list(reversed(word_count.most_common()))[:border]

[('man', 1), ('artificial', 1), ('civitas', 1), ('--(', 1), ('state', 1)]

In [40]:
stopwords = word_count.most_common()[:border] + list(reversed(word_count.most_common()))[:border]
stopwords

[(',', 96),
 ('.', 66),
 ('the', 58),
 ('of', 36),
 ('and', 35),
 ('man', 1),
 ('artificial', 1),
 ('civitas', 1),
 ('--(', 1),
 ('state', 1)]

In [41]:
stopwords = [s[0] for s in stopwords]

In [42]:
stopwords

[',', '.', 'the', 'of', 'and', 'man', 'artificial', 'civitas', '--(', 'state']

## 3. Build vocab

In [43]:
len(set(flatten(corpus)))
len(set(stopwords))

10

In [44]:
vocab = list(set(flatten(corpus)) - set(stopwords))
vocab.append('<UNK>')

In [45]:
print(len(set(flatten(corpus))), len(vocab))

592 583


In [46]:
word2index = {'<UNK>' : 0}

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
index2word = {v:k for k, v in word2index.items()}
index2word

{0: '<UNK>',
 1: 'nuee',
 2: 'thought',
 3: 'into',
 4: 'etymology',
 5: 'cetus',
 6: 'paunch',
 7: 'mockingly',
 8: 'bulk',
 9: 'we',
 10: 'towards',
 11: 'take',
 12: 'from',
 13: 'men',
 14: 'scarcely',
 15: 'e',
 16: 'brought',
 17: 'storied',
 18: 'including',
 19: 'spermacetti',
 20: 'noble',
 21: 'somehow',
 22: 'latin',
 23: 'six',
 24: 'verbal',
 25: ']',
 26: 'perisheth',
 27: 'certain',
 28: 'through',
 29: 'lucian',
 30: 'after',
 31: 'sore',
 32: 'ignorance',
 33: 'lost',
 34: 'english',
 35: 'days',
 36: 'greek',
 37: 'annals',
 38: 'far',
 39: 'henry',
 40: 'generally',
 41: 'bird',
 42: 'threatens',
 43: 'hvalt',
 44: 'that',
 45: 'no',
 46: 'this',
 47: 'yards',
 48: 'justly',
 49: 'plainly',
 50: 'late',
 51: 'arched',
 52: 'gay',
 53: 'mortality',
 54: 'proceeded',
 55: 'icelandic',
 56: 'strike',
 57: '-',
 58: 'together',
 59: 'nations',
 60: 'chaos',
 61: 'school',
 62: 'doubt',
 63: 'whatever',
 64: 'dan',
 65: 'deep',
 66: 'reminded',
 67: 'whose',
 68: 'peacefu

## 4. Prepare train data

In [47]:
WINDOW_SIZE = 3
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])
windows

[('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by'),
 ('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman'),
 ('<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville'),
 ('[', 'moby', 'dick', 'by', 'herman', 'melville', '1851'),
 ('moby', 'dick', 'by', 'herman', 'melville', '1851', ']'),
 ('dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>'),
 ('by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>'),
 ('herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>', '<DUMMY>'),
 ('<DUMMY>', '<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>'),
 ('<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>', '<DUMMY>'),
 ('<DUMMY>', '<DUMMY>', '<DUMMY>', '(', 'supplied', 'by', 'a'),
 ('<DUMMY>', '<DUMMY>', '(', 'supplied', 'by', 'a', 'late'),
 ('<DUMMY>', '(', 'supplied', 'by', 'a', 'late', 'consumptive'),
 ('(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher'),
 ('supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to'),
 ('by', 'a', 'late', 'c

In [48]:
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>':
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))
train_data

[('[', 'moby'),
 ('[', 'dick'),
 ('[', 'by'),
 ('moby', '['),
 ('moby', 'dick'),
 ('moby', 'by'),
 ('moby', 'herman'),
 ('dick', '['),
 ('dick', 'moby'),
 ('dick', 'by'),
 ('dick', 'herman'),
 ('dick', 'melville'),
 ('by', '['),
 ('by', 'moby'),
 ('by', 'dick'),
 ('by', 'herman'),
 ('by', 'melville'),
 ('by', '1851'),
 ('herman', 'moby'),
 ('herman', 'dick'),
 ('herman', 'by'),
 ('herman', 'melville'),
 ('herman', '1851'),
 ('herman', ']'),
 ('melville', 'dick'),
 ('melville', 'by'),
 ('melville', 'herman'),
 ('melville', '1851'),
 ('melville', ']'),
 ('1851', 'by'),
 ('1851', 'herman'),
 ('1851', 'melville'),
 ('1851', ']'),
 (']', 'herman'),
 (']', 'melville'),
 (']', '1851'),
 ('etymology', '.'),
 ('.', 'etymology'),
 ('(', 'supplied'),
 ('(', 'by'),
 ('(', 'a'),
 ('supplied', '('),
 ('supplied', 'by'),
 ('supplied', 'a'),
 ('supplied', 'late'),
 ('by', '('),
 ('by', 'supplied'),
 ('by', 'a'),
 ('by', 'late'),
 ('by', 'consumptive'),
 ('a', '('),
 ('a', 'supplied'),
 ('a', 'by'),
 (

In [49]:
X_p = []
y_p = []

In [50]:
for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))

In [51]:
train_data = list(zip(X_p, y_p))

In [52]:
a = [1,2,3]
b = [4,5,6]
a+ b

[1, 2, 3, 4, 5, 6]

In [53]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        self.embedding_v.weight.data.uniform_(-1, 1) # init
        self.embedding_u.weight.data.uniform_(0, 0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
    def forward(self, center_words,target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV
        
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) # log-softmax
        
        return nll # negative log likelihood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds 

## 5. Train

In [54]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100

In [55]:
losses = []
model = Skipgram(len(word2index), EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [62]:
loss.data.tolist()

6.368189811706543

In [63]:
for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) # B x 1
        targets = torch.cat(targets) # B x 1
        vocabs = prepare_sequence(list(vocab), word2index).expand(inputs.size(0), len(vocab))  # B x V
        model.zero_grad()

        loss = model(inputs, targets, vocabs)
        
        loss.backward()
        optimizer.step()
   
        losses.append(loss.data.tolist())

    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses = []

Epoch : 0, mean_loss : 6.17
Epoch : 10, mean_loss : 4.37
Epoch : 20, mean_loss : 3.48
Epoch : 30, mean_loss : 3.31
Epoch : 40, mean_loss : 3.26
Epoch : 50, mean_loss : 3.24
Epoch : 60, mean_loss : 3.22
Epoch : 70, mean_loss : 3.21
Epoch : 80, mean_loss : 3.21
Epoch : 90, mean_loss : 3.20


In [64]:
def word_similarity(target, vocab):
    if USE_CUDA:
        target_V = model.prediction(prepare_word(target, word2index))
    else:
        target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: continue
        
        if USE_CUDA:
            vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        else:
            vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0] 
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10] # sort by similarity

In [67]:
test = random.choice(list(vocab))
test

'eyes'

In [68]:
word_similarity(test, vocab)

[['empty', 0.8058958649635315],
 ['full', 0.7151128649711609],
 ['fegee', 0.6264484524726868],
 ['ponderous', 0.5834770202636719],
 ['t', 0.5713217854499817],
 ['raising', 0.5608559846878052],
 ['his', 0.5588749051094055],
 ['etymology', 0.5560470819473267],
 ['greek', 0.5502840280532837],
 ['erromangoan', 0.5272930264472961]]